#Preprocessing

Load dataset

In [3]:
!wget "https://github.com/danglive/Segmentation-using-Unet/raw/master/nails.tar.gz"

--2019-11-16 18:39:10--  https://github.com/danglive/Segmentation-using-Unet/raw/master/nails.tar.gz
Resolving github.com (github.com)... 192.30.253.113
Connecting to github.com (github.com)|192.30.253.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/danglive/Segmentation-using-Unet/master/nails.tar.gz [following]
--2019-11-16 18:39:10--  https://raw.githubusercontent.com/danglive/Segmentation-using-Unet/master/nails.tar.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19094516 (18M) [application/octet-stream]
Saving to: ‘nails.tar.gz’

nails.tar.gz        100%[===================>]  18.21M  74.9MB/s    in 0.2s    

2019-11-16 18:39:11 (74.9 MB/s) - ‘nails.tar.gz’ saved [19094516/19094516]



In [0]:
import numpy as np
import tarfile
import matplotlib.pyplot as plt
import os
from PIL import Image
from sklearn.model_selection import train_test_split
import keras
from keras.models import Model
from keras.layers import Conv2D, MaxPooling2D, Input, Conv2DTranspose, Concatenate
from keras.optimizers import Adam, SGD
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from keras import backend as K
from keras.utils import plot_model
import tensorflow as tf
from IPython.display import clear_output

In [0]:
class preprocess:
    
    epochs = 50
    batch_size = 4
    val_split = .1
    sz = (192, 160)
    
    def __init__(self, dataset):
        if len(dataset):
            tar = tarfile.open(dataset)
            tar.extractall()
            tar.close()
        mask_files = os.listdir('nails/mask')
        raw_files = os.listdir('nails/raw')
        # find intersection of two lists
        files = list(set(raw_files).intersection(mask_files))
        test_files = list(set(mask_files).symmetric_difference(raw_files))
        X_train = []
        X_test = []
        y = []
        for f in files:
            mask = Image.open('nails/mask/'+f)
            mask = np.array(mask.resize(self.sz))
            mask = mask.mean(axis=2)
            mask[mask<250]=0
            mask[mask>0]=1
            if not mask.sum():
                continue
            y.append(mask)
            raw = Image.open('nails/raw/'+f)
            raw = np.array(raw.resize(self.sz))
            X_train.append(raw)
        for f in test_files:
            try:
                raw = Image.open('nails/raw/'+f)
            except:
                continue
            raw = np.array(raw.resize(self.sz))
            X_test.append(raw)
        X_train = np.array(X_train).astype('float32')
        X_train /= 255
        X_test = np.array(X_test).astype('float32')
        X_test /= 255
        self.X_test = X_test
        self.X_train = X_train
        y = np.array(y)
        self.y = np.expand_dims(y,3)
        self.sz = self.sz[::-1] + (3,)

In [0]:
dataset = 'nails.tar.gz'

In [0]:
epochs = 50
batch_size = 4
val_split = .1
sz = (192, 160)

In [43]:
data=preprocess(dataset)
X_train = data.X_train
X_test = data.X_test
X_train.shape,X_test.shape

((192, 160, 192, 3), (21, 160, 192, 3))